In [1]:
"""
Author: Trenz Electronic GmbH / Kilian Jahn 27.09.2019 - edited 23.10.2019
Python version:   3.7.3

This notebook is an example on using the ADC of the Trenz modules TEI0015
and TEI0016. The notebook features data acquisition, conversion to 
standard integer and a Fourier transformation.
Further explonations are available in the Trenz Electronic wiki for the above
mentioned modules.
"""


import serial  # Serial/Comport connection
import serial.tools.list_ports

from matplotlib import pyplot as plt  # Interactive Graphs
import numpy as np

import matplotlib.patches as patches # Plot legend

import ipywidgets as widgets  # GUI
from IPython.display import clear_output  # Clearing plots/output

import TEI001xCodeModule



# Global variables------------------------------------------------------------------------------------------

# Set default values for comport and module here ##############
#
# No default comport = 'COMx' -> manually select port, if desired comport is for example 4 set to 'COM4'
comport = 'COMx'
# 
#
# #############################################################


# Variables for interaction with the GUI
moduleID = 0
moduleSamplingRate = 0
moduleAdcLinearGain = 1
moduleAdcMaxAbsThreshold = 0
moduleSignalGeneratorON = False
signalSampleLength = 16
plotSignalScale = 10


# GUI function ---------------------------------------------------------------------------------------------

# This function contains all the code for data collection from the module and plotting it
def performDataAcquisitionFFT(handleButton):
    global moduleID
    global moduleSamplingRate
    global handleMessageBox
    global moduleAdcLinearGain
    global moduleSignalGeneratorON
    
    clear_output() # Clear the plots from a previous program run and restore the GUI
    display(handleMessageBox, handleSerial, handleGain, handleSamples, handleSquareWave, handleButton)
    try:# Prevent printing into the text field
        sliderGain(moduleAdcLinearGain, False)
        squareWaveSignal(moduleSignalGeneratorON, False)
        # Collect  adc data and convert to voltage
        adcSignals = TEI001xCodeModule.dataCollect(comport, signalSampleLength, moduleID)
        # Activating interactive graph plots
        %matplotlib notebook
        plt.ion()
        
        # Process data for plotting, convert from samples to Milli Seconds
        adcSignalslength = len(adcSignals[0])
        periode = 1000 * 1 / moduleSamplingRate
        duration = np.arange(0, adcSignalslength * periode, periode)
        # Scale the signal acording to the gain
        adcSignals[0][:] = [x / moduleAdcLinearGain for x in adcSignals[0]]
        # Plot the adc voltage data
        handlePlot = TEI001xCodeModule.plottingGraphs(0, 
                    'Timedomain of adc signal', 'time [$ms$]', 'ADC measurement range [$V$]', 
                    [duration, adcSignals[0]], [0, duration[-1], -plotSignalScale, plotSignalScale])        
        messageTextBoxAdd("Data captured")
        
        # Check if measurment limits are exceeded
        adcLimitsExceeded = TEI001xCodeModule.signalLimitsExceed(adcSignals[2], moduleAdcMaxAbsThreshold)
        if adcLimitsExceeded > 10:
            text = 'WARNING - ' + str(adcLimitsExceeded) + ' samples exceed measurement range'
            handleAxes = plt.gca()            
            handlePlot.text(0.5, 0.5, text, color = 'red', backgroundcolor='white',
                     transform = handleAxes.transAxes, horizontalalignment = 'center')
            messageTextBoxAdd("WARNING - INPUTSIGNAL EXCEDDS MEASURMENT LIMITS !")
        
        # Convert adc data to a FFT. Return value is: [frequencies, fftSignal]
        temp = TEI001xCodeModule.performeFFTdbFS(moduleSamplingRate, adcSignals[1])        
        # Process data for plotting, convert to Kilo Hertz
        minimum = min(temp[1])
        temp[0][:] = [y / 1000 for y in temp[0]]
        messageTextBoxAdd("Fast Fourier Transformation done")
        # Plot the FFT
        TEI001xCodeModule.plottingGraphs(1, 'Signal spectrum', 'Frequency [$kHz$]', 'Amplitude [$dbFS$]', 
                                         temp, [0, 0, int(minimum * 1.01-3.5), 0])
    except:
        messageTextBoxAdd("Runtime error performing adc data collection")


# The following functions are for the ease of use-----------------------------------------------------------
        
def sliderGain(gain, printMessage = True):
    global comport
    global plotSignalScale
    global moduleAdcLinearGain
    
    try:
        TEI001xCodeModule.sendCommand(comport, str(gain))
        moduleAdcLinearGain = int(gain)
        # Define plot limits acording to the gain
        plotSignalScale = 10 / moduleAdcLinearGain
        messageTextBoxAdd("Set ADC gain to " + str(moduleAdcLinearGain) 
                     + " or Measurement range to +/- " + str(plotSignalScale) + " V", printMessage)
    except:
        messageTextBoxAdd("Error setting gain")

    
def sliderSamples(guiSamples):
    global signalSampleLength
    
    signalSampleLength = int(guiSamples)
    messageTextBoxAdd("Set sample length to " + str(guiSamples) + " kS")
    
    
def selectModule(selection):
    global moduleSamplingRate
    global moduleID
    global moduleAdcMaxAbsThreshold
    modulesmoduleSamplingRate = [2000000, 500000, 1000000]
    moduleName = ["TEI0015 - 2 MS", "TEI0016A - 0.5 MS", "TEI0016B - 1 MS"]
    moduleAdcMaxInt = [131072, 32768, 32768]
    maxIntThreshold = [20, 5, 5] 
        
    moduleSamplingRate = modulesmoduleSamplingRate[selection]
    messageTextBoxAdd("Detected module (ID " + str(selection + 1) + ") - " + moduleName[selection] + "/s")
    moduleAdcMaxAbsThreshold = moduleAdcMaxInt[selection] - maxIntThreshold[selection]
    

def selectComport(selection):
    global comport
    global guiDisabled    
    global moduleAdcLinearGain
    global moduleID
        
    # Cut from selection the comport
    for index, character in enumerate(selection):
        if character == ' ':
            comport = str(selection[:int(index)])
            break    
    messageTextBoxAdd(comport + " selected\n")
    
    moduleID = TEI001xCodeModule.getModuleId(comport)
    if moduleID == 1 or moduleID == 2 or moduleID == 3:
        selectModule(moduleID - 1)
        sliderGain(moduleAdcLinearGain)
        # Enable the gui
        handleGain.disabled = False
        handleSamples.disabled = False
        handleSquareWave.disabled = False
        handleButton.disabled = False
    else:
        # Disable the gui        
        handleGain.disabled = True
        handleSamples.disabled = True
        handleSquareWave.disabled = True
        handleButton.disabled = True
        messageTextBoxAdd("No module found on selected comport")
    
    
def squareWaveSignal(marker, printMessage = True):
    global moduleSignalGeneratorON
    
    try:
        if marker == True:
            TEI001xCodeModule.sendCommand(comport, "F")
            moduleSignalGeneratorON = True
            messageTextBoxAdd("Square wave generator ON [10 kHz, +/- 3.3 V]", printMessage)
        else:
            TEI001xCodeModule.sendCommand(comport, "f")
            moduleSignalGeneratorON = False
            messageTextBoxAdd("Square wave generator OFF [10 kHz, +/- 3.3 V]", printMessage)
    except:
        messageTextBoxAdd("Error, changing square wave signal")
        
        
def messageTextBoxAdd(text, printMessage = True):
    global handleMessageBox
    
    if printMessage == True:
        handleMessageBox.value = text + "\n" + handleMessageBox.value


# Programm starts here--------------------------------------------------------------------------------------

# Define GUI elements. Consider default comport, if defined.
optionList = []
optionList.append("Select comport")  
if comport != 'COMx': # No default comport set
    optionList.append(comport + " Default comport")      
ports = serial.tools.list_ports.comports() # Gather available comports
for port, desc, hwid in sorted(ports):
    temp = port + " \t" + desc + " \t" + hwid
    optionList.append(temp)
handleSerial = widgets.Dropdown(options = optionList, description = 'Comport')
                                
handleGain = widgets.SelectionSlider(value = '1', description='Gain linear', 
                        options=['1', '2', '4', '8'], disabled = True)
handleSamples = widgets.SelectionSlider(description='Samples [kS]', 
                        options=['16', '32', '64', '128', '256', '512', '1024'], disabled = True)
handleSquareWave = widgets.Checkbox(value=False, description='Activate square wave signal', disabled = True)
handleButton = widgets.Button(description="Capture FFT Data", disabled = True)
handleButton.on_click(performDataAcquisitionFFT)
if comport == 'COMx': # No default comport set
    firstText = "Select comport"
else:
    firstText = comport + " As default selected"
handleMessageBox = widgets.Textarea(firstText, layout=widgets.Layout(width='50%', height='100%'))


# Bind GUI elements to their functions
widgets.interactive(selectComport, selection = handleSerial)
widgets.interactive(sliderGain, gain = handleGain)
widgets.interactive(sliderSamples, guiSamples = handleSamples)
widgets.interactive(squareWaveSignal, marker = handleSquareWave)


# Show the GUI
display(handleMessageBox, handleSerial, handleGain, handleSamples, handleSquareWave, handleButton)


# Initialize Notebook to default comport
if comport != 'COMx':
    handleSerial.index = 1


Textarea(value='Fast Fourier Transformation done\nData captured\nSquare wave generator ON [10 kHz, +/- 3.3 V]\…

Dropdown(description='Comport', index=4, options=('Select comport', 'COM1 \tKommunikationsanschluss (COM1) \tA…

SelectionSlider(description='Gain linear', options=('1', '2', '4', '8'), value='1')

SelectionSlider(description='Samples [kS]', options=('16', '32', '64', '128', '256', '512', '1024'), value='16…

Checkbox(value=True, description='Activate square wave signal')

Button(description='Capture FFT Data', style=ButtonStyle())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>